<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Use SPSS to predict customer churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook shows you how to deploy a sample SPSS stream, and start scoring new data. 

Some familiarity with bash is helpful. This notebook uses Python 3.

In this notebook, you will import and use a model that was trained on a **Telco Customer Churn** data set, which contains anonymous data about customers of a telecommunication company. Use the details of this data set to predict customer churn, which is critical information for a business because it's easier to retain existing customers than to acquire new ones.

## Learning goals

In this notebook you learn how to:

-  Work with a Watson Machine Learning instance
-  Deploy the SPSS model online
-  Use the deployed model to score data


## Contents

1.	[Set up the environment](#setup)
2.	[Work with the WML service instance](#instance) 
3.	[Persist, deploy and score in the IBM Cloud](#scoring)
4.	[Summary](#summary)

<a id="setup"></a>
## 1. Set up

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance is <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">here</a>).

-   Install <a href="https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search" target="_blank" rel="noopener no referrer">watson-machine-learning-client</a> (not required on Watson Studio).

Install the Python API client from pypi.

In [ ]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install watson-machine-learning-client --upgrade

<a id="instance"></a>
## 2. Work with the WML service instance

Authenticate to the Watson Machine Learning service on IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the [Service Credentials](https://console.bluemix.net/docs/services/service_credentials.html#service_credentials) tab of the service instance that you created on IBM Cloud. <BR>If you cannot see the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [1]:
wml_credentials = {
  "instance_id": "***",
  "password": "***",
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "***"
}

Import the `watson-machine-learning-client` and authenticate to the service instance.

In [2]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2018-10-01 22:10:33,104 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [3]:
client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your Watson Machine Learning service are available in ``service_instance.details``.

In [4]:
instance_details = client.service_instance.details

Get a list of stored models for the WML instance.

In [5]:
client.repository.list_models()

----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


Get a list of deployed models.

In [6]:
client.deployments.list()

----  ----  ----  -----  -------  ---------  -------------
GUID  NAME  TYPE  STATE  CREATED  FRAMEWORK  ARTIFACT TYPE
----  ----  ----  -----  -------  ---------  -------------


<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will learn how to:
- [3.1 Download and save an SPSS sample model](#save)
- [3.2 Use the WML REST API to create an online deployment](#create)
- [3.3 Score a new data record](#score)

For more information about REST APIs, see the <a href="http://watson-ml-api.mybluemix.net/" target="_blank" rel="noopener no referrer">Swagger Documentation</a>.

### 3.1 Download and save the SPSS model<a id="save"></a>

Download the SPSS sample model from the Git project.

**Note:** You may need to install the `wget` package. To do this run: 

In [7]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [8]:
# Download the sample SPSS model.
import os, wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the SPSS sample model to your WML instance.

In [9]:
# Save the SPSS model to the WML repository.
props_spss = {client.repository.ModelMetaNames.NAME: 'SPSS model for Churn prediction',
              client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              client.repository.ModelMetaNames.FRAMEWORK_NAME: "spss-modeler", 
              client.repository.ModelMetaNames.FRAMEWORK_VERSION: "18.1"}

model_details = client.repository.store_model(filename, props_spss)

Get details of all the models saved in the repository.

In [10]:
# List models in the repository.
print(model_details)

{'entity': {'runtime_environment': 'None Provided', 'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/published_models/704a23ce-711f-40f6-be0c-a289df74dbe8/learning_configuration', 'name': 'SPSS model for Churn prediction', 'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/published_models/704a23ce-711f-40f6-be0c-a289df74dbe8/feedback', 'model_type': 'spss-modeler-18.1', 'latest_version': {'guid': 'bc5d186f-d762-485a-a375-0c1a468d4dbd', 'created_at': '2018-10-01T22:10:33.455Z', 'url': 'https://us-south.ml.cloud.ibm.com/v3/ml_assets/models/704a23ce-711f-40f6-be0c-a289df74dbe8/versions/bc5d186f-d762-485a-a375-0c1a468d4dbd'}, 'evaluation_metrics_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/published_models/704a23ce-711f-40f6-be0c-a289df74dbe8/evaluation_metrics', 'author': {'name': 'IBM'}, 'deployments': {'url': 'http

### 3.2 Create the online deployment<a id="create"></a>

Use the commands below to create the online deployment.

In [11]:
model_uid = client.repository.get_model_uid(model_details)
model_deployment = client.deployments.create(model_uid, "Sample SPSS model deployment")



#######################################################################################

Synchronous deployment creation for uid: '704a23ce-711f-40f6-be0c-a289df74dbe8' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS...........
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='25df1087-19ea-4c40-a550-366ad8fe3430'
------------------------------------------------------------------------------------------------




Now check details about your deployments.

In [12]:
deployment_uid = client.deployments.get_uid(model_deployment)
print(client.deployments.get_details(deployment_uid))

{'entity': {'runtime_environment': 'None Provided', 'model_type': 'spss-modeler-18.1', 'description': 'Description of deployment', 'name': 'Sample SPSS model deployment', 'type': 'online', 'deployed_version': {'guid': 'bc5d186f-d762-485a-a375-0c1a468d4dbd', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/704a23ce-711f-40f6-be0c-a289df74dbe8/versions/bc5d186f-d762-485a-a375-0c1a468d4dbd'}, 'status': 'DEPLOY_SUCCESS', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'scoring_url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/deployments/25df1087-19ea-4c40-a550-366ad8fe3430/online', 'deployable_asset': {'guid': '704a23ce-711f-40f6-be0c-a289df74dbe8', 'created_at': '2018-10-01T22:11:47.168Z', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/published_models/704a23ce-711f-40f6-be0c-a289df74dbe8', 'name': 'SPSS model for Churn prediction', 'type': 'model'}}, 'metadata': {'guid': '25d

### 3.3 Score a new data record<a id="score"></a>

Use the following commands to get the scoring endpoint, `scoring_url`, for your deployed model.

In [13]:
# Get scoring endpoint.
scoring_url = client.deployments.get_scoring_url(model_deployment)
print(scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/d0755448-6eb4-425a-b35b-479be91ff2d5/deployments/25df1087-19ea-4c40-a550-366ad8fe3430/online


Use the client `score` method to score data about a customer.

In [14]:
scoring_payload = {"fields":["customerID","gender","SeniorCitizen","Partner","Dependents","tenure","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","MonthlyCharges","TotalCharges","Churn","SampleWeight"],"values":[["3638-WEABW","Female",0,"Yes","No",58,"Yes","Yes","DSL","No","Yes","No","Yes","No","No","Two year","Yes","Credit card (automatic)",59.9,3505.1,"No",2.768]]}
prediction = client.deployments.score(scoring_url, scoring_payload)

print(prediction)

{'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]], 'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn']}


As you can see this sample telco customer is satisfied ("Predicted Churn", "No").

Now you can see your model in the list of stored models for the WML instance.

In [15]:
client.repository.list_models()

------------------------------------  -------------------------------  ------------------------  -----------------
GUID                                  NAME                             CREATED                   FRAMEWORK
704a23ce-711f-40f6-be0c-a289df74dbe8  SPSS model for Churn prediction  2018-10-01T22:10:33.398Z  spss-modeler-18.1
------------------------------------  -------------------------------  ------------------------  -----------------


Get a list of deployed models.

In [16]:
client.deployments.list()

------------------------------------  ----------------------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                          TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
25df1087-19ea-4c40-a550-366ad8fe3430  Sample SPSS model deployment  online  DEPLOY_SUCCESS  2018-10-01T22:11:47.249Z  spss-modeler-18.1  model
------------------------------------  ----------------------------  ------  --------------  ------------------------  -----------------  -------------


<a id="summary"></a>
## 4. Summary     

You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. 

Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener noreferrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts.

### Author

**Lukasz Cmielowski, PhD,** is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.

Copyright © 2017, 2018 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>